<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 36px">Notebook de exemplo para como corretamente fazer o ETL dos dados - MS</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 22px; padding: 5px; border-radius: 5px;">
  <li>Apresenta apenas como organizar os dados com um exemplo em JSON.</li>
  <li>Quais devem ser os atributos para envio no Postgre DB.</li>
</ul>

In [10]:
import pandas as pd
import numpy as np
import os, json
import itertools

In [ ]:
# Importando os dados de exemplo do calendario
# Obviamente esses dados serão extraídos diretamente da chamada da API, aqui é apenas um exemplo.

with open('mscalendar_data_example.json', 'r') as file:
    # Coletando os dados do arquivo local
    data = json.load(file)
    result_calendar = []
    # Esses nomes de atributo "name_" são ficticios e apenas como sinalização de exemplo
    for nome in ['name_alice','name_bob']:
        # Aqui estamos acessando os dados de calendário de cada colaborador da empresa
        for meeting in data[nome]['value']:
            # Posteriormente coletamos os dados dos participantes no campo attendees
            participantes = [participante['emailAddress']['address'] for participante in meeting['attendees']]
            # E o organizador podemos coletar diretamente do organizer
            organizador = [meeting['organizer']['emailAddress']['address']]

            # Aqui nos fazemos um append de ambas as listas para fazer a combinação 2 a 2 de cada participante que estava no evento do calendário
            # Pode existir combinações organizador > participante como participante > participante. É importante existir ambas as relações     
            participantes = participantes + organizador
            combinations = list(itertools.combinations(participantes, 2))

            # Aqui criamos um Dict que será utilizados para criar um DataFrame posteriormente
            # iCalUId é o identificador único do evento do calendário entre todos os calendários de todos os usuários. NÃO UTILIZE O IDENTIFICADOR "ID"
            result_calendar.extend([
                {
                 'iCalUId': meeting['iCalUId'],
                 'sender':dupla[0],
                 'receiver':dupla[1],
                 'data': pd.Timestamp(meeting['start']['dateTime']).date(),
                 'de_onde':'calendar'
                }
                for dupla in combinations
            ])    

result_calendar = pd.DataFrame(result_calendar) ; result_calendar

,iCalUId,sender,receiver,data,de_onde
0,040000008200E00074C5B7101A82E00800000000A1A1A1A1,bob.smith@company.com,alice.johnson@company.com,2025-03-11,calendar
1,040000008200E00074C5B7101A82E00800000000B2B2B2B2,alice.johnson@company.com,charlie.brown@company.com,2025-03-13,calendar
2,040000008200E00074C5B7101A82E00800000000B2B2B2B2,alice.johnson@company.com,david.lee@company.com,2025-03-13,calendar
3,040000008200E00074C5B7101A82E00800000000B2B2B2B2,charlie.brown@company.com,david.lee@company.com,2025-03-13,calendar
4,040000008200E00074C5B7101A82E00800000000C3C3C3C3,alice.johnson@company.com,franklin.lewis@company.com,2025-03-15,calendar
5,040000008200E00074C5B7101A82E00800000000C3C3C3C3,alice.johnson@company.com,emily.davis@company.com,2025-03-15,calendar
6,040000008200E00074C5B7101A82E00800000000C3C3C3C3,franklin.lewis@company.com,emily.davis@company.com,2025-03-15,calendar
7,040000008200E00074C5B7101A82E00800000000A1A1A1A1,bob.smith@company.com,alice.johnson@company.com,2025-03-12,calendar


<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, temos duas vezes o final A1A1A1 no iCalUId (Primeira e ultima linha do DF) aparecendo para exatamente sender e receiver iguais.</li>
  <li>Isso acontece pois extraímos os dados de dois calendários de users diferentes e por isso precisamos eliminar essa duplicação para isso criamos um hash_id que mapeia a relação</li>
</ul>

In [ ]:
# Utilize dessa função para criar o hashmap dos colaboradores da organização
# Queremos mapear a relação entre 2 colaboradores independente da ordem das colunas
import hashlib
def generate_hash(sender, receiver):
    interaction = f"{min(sender, receiver)}-{max(sender, receiver)}"
    return hashlib.sha256(interaction.encode()).hexdigest()

result_calendar['hash_id'] = result_calendar.apply(lambda row: generate_hash(row['sender'], row['receiver']), axis=1) ; result_calendar

,iCalUId,sender,receiver,data,de_onde,hash_id
0,040000008200E00074C5B7101A82E00800000000A1A1A1A1,bob.smith@company.com,alice.johnson@company.com,2025-03-11,calendar,43cc8c9abcb9beca5d3fa06e0645ec9cbe4802b6d46e40...
1,040000008200E00074C5B7101A82E00800000000B2B2B2B2,alice.johnson@company.com,charlie.brown@company.com,2025-03-13,calendar,35431b10acc26241339c796f8d3c67fc97add2a71c0fcd...
2,040000008200E00074C5B7101A82E00800000000B2B2B2B2,alice.johnson@company.com,david.lee@company.com,2025-03-13,calendar,2eff1241584d53d95c9a8642ada2d9a2d2d7586a3477b8...
3,040000008200E00074C5B7101A82E00800000000B2B2B2B2,charlie.brown@company.com,david.lee@company.com,2025-03-13,calendar,207f8e81a5365710ac53a2737bae71efc8a1cdfc635cb1...
4,040000008200E00074C5B7101A82E00800000000C3C3C3C3,alice.johnson@company.com,franklin.lewis@company.com,2025-03-15,calendar,6b1fefaed172e4b5e8c3b8bee785ae5ea40182cc187d27...
5,040000008200E00074C5B7101A82E00800000000C3C3C3C3,alice.johnson@company.com,emily.davis@company.com,2025-03-15,calendar,04ff6ff1af11efd6a8dfb4cca43f330a2cda604a2bb2e1...
6,040000008200E00074C5B7101A82E00800000000C3C3C3C3,franklin.lewis@company.com,emily.davis@company.com,2025-03-15,calendar,d5e7af97109bae7380f44c5793d7d4c50b9d098003db6f...
7,040000008200E00074C5B7101A82E00800000000A1A1A1A1,bob.smith@company.com,alice.johnson@company.com,2025-03-12,calendar,43cc8c9abcb9beca5d3fa06e0645ec9cbe4802b6d46e40...


In [14]:
result_calendar[result_calendar['iCalUId'] == '040000008200E00074C5B7101A82E00800000000A1A1A1A1']['hash_id'].value_counts()

43cc8c9abcb9beca5d3fa06e0645ec9cbe4802b6d46e40b463deb53ad4a47ab9    2
Name: hash_id, dtype: int64

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, como realmente para um iCalUId temos 2 hash_ids iguais, precisamos agrupar e eliminar duplicatas</li>
</ul>

In [ ]:
result_calendar.groupby(['iCalUId','hash_id']).agg(
    sender=('sender','max'),
    receiver=('receiver','max'),
    data=('data','max'),
    de_onde=('de_onde','max')
).reset_index()

,iCalUId,hash_id,sender,receiver,data,de_onde
0,040000008200E00074C5B7101A82E00800000000A1A1A1A1,43cc8c9abcb9beca5d3fa06e0645ec9cbe4802b6d46e40...,bob.smith@company.com,alice.johnson@company.com,2025-03-12,calendar
1,040000008200E00074C5B7101A82E00800000000B2B2B2B2,207f8e81a5365710ac53a2737bae71efc8a1cdfc635cb1...,charlie.brown@company.com,david.lee@company.com,2025-03-13,calendar
2,040000008200E00074C5B7101A82E00800000000B2B2B2B2,2eff1241584d53d95c9a8642ada2d9a2d2d7586a3477b8...,alice.johnson@company.com,david.lee@company.com,2025-03-13,calendar
3,040000008200E00074C5B7101A82E00800000000B2B2B2B2,35431b10acc26241339c796f8d3c67fc97add2a71c0fcd...,alice.johnson@company.com,charlie.brown@company.com,2025-03-13,calendar
4,040000008200E00074C5B7101A82E00800000000C3C3C3C3,04ff6ff1af11efd6a8dfb4cca43f330a2cda604a2bb2e1...,alice.johnson@company.com,emily.davis@company.com,2025-03-15,calendar
5,040000008200E00074C5B7101A82E00800000000C3C3C3C3,6b1fefaed172e4b5e8c3b8bee785ae5ea40182cc187d27...,alice.johnson@company.com,franklin.lewis@company.com,2025-03-15,calendar
6,040000008200E00074C5B7101A82E00800000000C3C3C3C3,d5e7af97109bae7380f44c5793d7d4c50b9d098003db6f...,franklin.lewis@company.com,emily.davis@company.com,2025-03-15,calendar


### Critérios de aceite:

* id não é o identificador único entre todas as agendas, o iCalUId que realiza esse papel.
* É possível existir iCalUId duplicados.
* É possível existir hash_id duplicados.
* Não se pode possuir iCalUId e hash_id duplicados.
* É necessário realizar as combinações par a par de todos os participantes da reunião utilizando o itertools como foi demonstrado.